## Your Model 🌱Garden🌱 Execution Environment

Use this notebook to write a function that executes your model(s). Tag that function with the `@garden_pipeline` decorator.

Garden will take this notebook and build a container with it. When Garden executes your `@garden_pipeline`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@garden_pipeline`.

In [ ]:
!pip install git+https://github.com/Garden-AI/garden.git@a33dae1226ba258f27f6fbcc76c3d120f22c9a34

In [2]:
from garden_ai.model_connectors import HFConnector
from garden_ai import PipelineMetadata, garden_pipeline

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [ ]:
my_hugging_face_repo = HFConnector("willengler-uc/torch-mnist")

### Pipeline metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this PipelineMetadata object to describe your function.


In [5]:
my_pipeline_meta = PipelineMetadata(
    title="MNIST Digit Classsifier",
    description="Example of a PyTorch model in Garden, using the MNIST digit classification problem",
    authors=["Will Engler"],
)

### Helper Functions

Define any helper functions you need and use them in the function you want to let people run remotely

In [10]:
def get_a_test_image_batch():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
    data, target = next(iter(test_loader))
    return data, target

### Write your pipeline function that will run remotely

The `@garden_pipeline` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your pipeline metadata,
- connectors for any models you're using,
- the DOI of the garden you want this pipeline to be found in. (Check `garden-ai garden list` for the DOIs of your gardens.)

In [15]:
@garden_pipeline(metadata=my_pipeline_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/crbc-v840")
def classify_digits_mnist(image_batch):    
    download_path = my_hugging_face_repo.stage()
    from mnist_model import Net
    model = Net()
    model.load_state_dict(torch.load(download_path + "/mnist_model.pt"))
    model.eval()

    with torch.no_grad():
        output = model(image_batch)
        as_digits = output.data.max(1)[1]

    return as_digits

### Test your pipeline function

Finally, make sure your `@garden_pipeline` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@garden_pipeline` in the context of the notebook.

If you can hit "Kernel" -> "Restart and run all cells" and your test below works, your `@garden_pipeline` will work in your garden!


In [17]:
def test_pipeline():
    test_image_batch, expected_result_batch = get_a_test_image_batch()
    classified_digits = classify_digits(test_image_batch)
    print (f"Classified {len(classified_digits)} digits")
    print("Accuracy:", (classified_digits == expected_result_batch).sum()/len(classified_digits))

In [ ]:
#test_pipeline()